# AML Mini-Challenge - Credit Card Affinity Modelling

> Dominik Filliger & Noah Leuenberger

The task can be found [here](https://spaces.technik.fhnw.ch/storage/uploads/spaces/82/exercises/20240218__AML_Trainingscenter_MiniChallenge_Kreditkarten_Aufgabenstellung-1708412668.pdf).

# Setup

In [ ]:
import pandas as pd
from datetime import datetime

import numpy as np
'''  '''
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()
#plt.style.use('seaborn-white')
#plt.style.use('ggplot')

## Helper Functions

In [ ]:
def plot_categorical_variables(df, categorical_columns, fill_na_value='NA'):
    """
    Plots count plots for categorical variables in a DataFrame, filling NA values with a specified string.
    
    Parameters:
    - df: pandas.DataFrame containing the data.
    - categorical_vars: list of strings, names of the categorical variables in df to plot.
    - fill_na_value: string, the value to use for filling NA values in the categorical variables.
    """
    # Fill NA values in the specified categorical variables
    for var in categorical_columns:
        if df[var].isna().any():
            df[var] = df[var].fillna(fill_na_value)

    total = float(len(df))
    fig, axes = plt.subplots(nrows=len(categorical_columns), figsize=(14, len(categorical_columns) * 4.5))

    if len(categorical_columns) == 1:  # If there's only one categorical variable, wrap axes in a list
        axes = [axes]

    for i, var in enumerate(categorical_columns):
        ax = sns.countplot(x=var, data=df, ax=axes[i], order=df[var].value_counts().index)

        axes[i].set_title(f'Distribution of {var}')
        axes[i].set_ylabel('Count')
        axes[i].set_xlabel(var)

        for p in ax.patches:
            height = p.get_height()
            ax.text(p.get_x() + p.get_width() / 2.,
                    height + 3,
                    '{:1.2f}%'.format((height / total) * 100),
                    ha="center")

    plt.tight_layout()
    plt.show()

def plot_numerical_distributions(df, numerical_columns, kde=True, bins=30):
    """
    Plots the distribution of all numerical variables in a DataFrame.
    
    Parameters:
    - df: pandas.DataFrame containing the data.
    """

    # Determine the number of rows needed for subplots based on the number of numerical variables
    nrows = len(numerical_columns)

    # Create subplots
    fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(8, 5 * nrows))

    if nrows == 1:  # If there's only one numerical variable, wrap axes in a list
        axes = [axes]

    for i, var in enumerate(numerical_columns):
        sns.histplot(df[var], ax=axes[i], kde=kde, bins=bins)
        axes[i].set_title(f'Distribution of {var}')
        axes[i].set_xlabel(var)
        axes[i].set_ylabel('Frequency')

    plt.tight_layout()
    plt.show()

def plot_date_monthly_counts(df, date_column, title):
    """
    Plots the monthly counts of a date column in a DataFrame.
    
    Parameters:
    - df: pandas.DataFrame containing the data.
    - date_column: string, name of the date column in df to plot.
    - title: string, title of the plot.
    """
    df[date_column] = pd.to_datetime(df[date_column])
    df['month'] = df[date_column].dt.to_period('M')

    monthly_counts = df['month'].value_counts().sort_index()
    monthly_counts.plot(kind='bar', figsize=(14, 6))
    plt.title(title)
    plt.xlabel('Month')
    plt.ylabel('Count')
    plt.show()

def add_percentage_labels(ax, hue_order):
    for p in ax.patches:
        height = p.get_height()
        width = p.get_width()
        x = p.get_x()
        y = p.get_y()
        label_text = f'{height:.1f}%'
        label_x = x + width / 2
        label_y = y + height / 2
        ax.text(label_x, label_y, label_text, ha='center', va='center', fontsize=9, color='white', weight='bold')

# Data Import & Wrangling

In [ ]:
def remap_values(df, column, mapping):
    # assert that all values in the column are in the mapping except for NaN
    assert df[column].dropna().isin(mapping.keys()).all()
    
    df[column] = df[column].map(mapping, na_action='ignore')
    return df

def map_empty_to_nan(df, column):
    if df[column].dtype != 'object':
        return df

    df[column] = df[column].replace(r'^\s*$', np.nan, regex=True)
    return df

def read_csv(file_path, sep=";", dtypes=None):
    df = pd.read_csv(file_path, sep=sep, dtype=dtypes)
    
    for col in df.columns:
        df = map_empty_to_nan(df, col)
        
    return df

## Accounts

In [ ]:
accounts = read_csv("data/account.csv")

# Translated frequency from Czech to English
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
accounts = remap_values(accounts, 'frequency', {
    "POPLATEK MESICNE": "MONTHLY_ISSUANCE",
    "POPLATEK TYDNE": "WEEKLY_ISSUANCE",
    "POPLATEK PO OBRATU": "ISSUANCE_AFTER_TRANSACTION"
})

accounts['date'] = pd.to_datetime(accounts['date'], format='%y%m%d')

accounts.rename(columns={'date': 'account_created',
                         'frequency': 'account_frequency'}, inplace=True)

accounts.info()

In [ ]:
# todo add some basic eda here
accounts.head()

In [ ]:
accounts.nunique()

In [ ]:
plot_categorical_variables(accounts, ['account_frequency'])

In [ ]:
plot_numerical_distributions(accounts, ['account_created'])

## Clients

In [ ]:
clients = read_csv("data/client.csv")

def parse_birth_number(birth_number):
    birth_number_str = str(birth_number)

    # Extract year, month, and day from birth number from string
    # according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
    year = int(birth_number_str[:2])
    month = int(birth_number_str[2:4])
    day = int(birth_number_str[4:6])

    # Determine sex based on month and adjust month for female clients
    # according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
    if month > 50:
        sex = "Female"
        month -= 50
    else:
        sex = "Male"

    # Validate date
    assert 1 <= month <= 12
    assert 1 <= day <= 31
    assert 0 <= year <= 99

    if month in [4, 6, 9, 11]:
        assert 1 <= day <= 30
    elif month == 2:
        assert 1 <= day <= 29
    else:
        assert 1 <= day <= 31

    # Assuming all dates are in the 1900s
    birth_date = datetime(1900 + year, month, day)
    return pd.Series([sex, birth_date])


clients[['sex', 'birth_date']] = clients['birth_number'].apply(parse_birth_number)

# Calculate 'age' assuming the reference year is 1999
clients['age'] = clients['birth_date'].apply(lambda x: 1999 - x.year)

# Drop 'birth_number' column as it is no longer needed
clients = clients.drop(columns=['birth_number'])

clients.info()

In [ ]:
# todo add some basic eda here
clients.head()

In [ ]:
clients.describe()

In [ ]:
plot_numerical_distributions(clients, ['birth_date', 'age'])

## Dispositions

In [ ]:
dispositions = read_csv("data/disp.csv")


dispositions.info()

In [ ]:
dispositions.head()

In [ ]:
dispositions.describe()

In [ ]:
plot_categorical_variables(dispositions, ['type'])

As the goal of this model is to address accounts and not client directly we will focus on the clients which own an account and focus solely on them. 

In [ ]:
dispositions = dispositions[dispositions['type'] == 'OWNER']

## Orders

In [ ]:
orders = read_csv("data/order.csv")

# Translated from Czech to English
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
orders = remap_values(orders, 'k_symbol', {
    "POJISTNE": "Insurance_Payment",
    "SIPO": "Household",
    "LEASING": "Leasing",
    "UVER": "Loan_Payment"
})

orders['account_to'] = orders['account_to'].astype('category')

orders = orders.rename(columns={'amount': 'debited_amount'})

orders.info()

In [ ]:
# todo add some basic eda here
orders.head()

In [ ]:
orders.describe()

In [ ]:
orders.nunique()

There appear to be as many order ids as there are rows. 

In [ ]:
plot_categorical_variables(orders, ['k_symbol', 'bank_to'])

In [ ]:
plot_numerical_distributions(orders, ['debited_amount'])

## Transactions

In [ ]:
# column 8 is the 'bank' column which contains NaNs and must be read as string
transactions = read_csv("data/trans.csv", dtypes={8: str})

transactions['date'] = pd.to_datetime(transactions['date'], format='%y%m%d')

# Translated type, operations and characteristics from Czech to English
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
transactions = remap_values(transactions, 'type', {
    "VYBER": "Withdrawal", # Also withdrawal as it is against the documentation present in the dataset
    "PRIJEM": "Credit",
    "VYDAJ": "Withdrawal"
})

transactions = remap_values(transactions, 'operation', {
    "VYBER KARTOU": "Credit Card Withdrawal",
    "VKLAD": "Credit in Cash",
    "PREVOD Z UCTU": "Collection from Another Bank",
    "VYBER": "Withdrawal in Cash",
    "PREVOD NA UCET": "Remittance to Another Bank"
})

transactions = remap_values(transactions, 'k_symbol', {
    "POJISTNE": "Insurance Payment",
    "SLUZBY": "Payment on Statement",
    "UROK": "Interest Credited",
    "SANKC. UROK": "Sanction Interest",
    "SIPO": "Household",
    "DUCHOD": "Old-age Pension",
    "UVER": "Loan Payment"
})

# Set the amount to negative for withdrawals and positive for credits
transactions['amount'] = np.where(transactions['type'] == "Credit", transactions['amount'], -transactions['amount'])

transactions.rename(columns={'type': 'transaction_type'}, inplace=True)

transactions.info()

In [ ]:
# todo add some basic eda here
transactions.head()

In [ ]:
transactions.describe()

In [ ]:
plot_categorical_variables(transactions, ['transaction_type', 'operation', 'k_symbol'])

In [ ]:
plot_numerical_distributions(transactions, ['date', 'amount', 'balance'])

Looking at the distributions of the transaction table we can see that the count of transactions per year increase over time. So we can conclude that the bank has a growing client base. 

However, the other plots are not very useful. For one the transaction amount seems to be very sparse, ranging from values between -80000 and 80000. 

The balance distribution also showcases that there are accounts with a negative balance after a transaction, which would only make sense if debt is also included in this value. 

According to description of the field balance: "balance after transaction"

In [ ]:
# Getting a list of unique years from the dataset
transactions['year'] = transactions['date'].dt.year
transactions['month'] = transactions['date'].dt.month

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


years = sorted(transactions['year'].unique())

# Creating a figure with subplots for each year: one row for each year with two plots (box plot and bar chart)
fig, axs = plt.subplots(len(years) * 2, 1, figsize=(15, 6 * len(years)), sharex=True, gridspec_kw={'height_ratios': [3, 1] * len(years)})

for i, year in enumerate(years):
    # Filter transactions for the current year
    yearly_transactions = transactions[transactions['year'] == year]

    # Preparing data for the box plot: a list of amounts for each month for the current year
    amounts_per_month_yearly = [yearly_transactions[yearly_transactions['month'] == month]['amount'] for month in range(1, 13)]

    # Preparing data for the bar chart for the current year
    monthly_summary_yearly = yearly_transactions.groupby('month').agg(TotalAmount=('amount', 'sum'), TransactionCount=('amount', 'count')).reset_index()

    # Box plot for transaction amounts by month for the current year
    axs[i*2].boxplot(amounts_per_month_yearly, patch_artist=True)
    # now with seaborn
    # sns.boxplot(data=yearly_transactions, x='month', y='amount', ax=axs[i*2])
    axs[i*2].set_title(f'Transaction Amounts Per Month in {year} (Box Plot)')
    axs[i*2].set_yscale('symlog')
    axs[i*2].set_ylabel('Transaction Amounts (log scale)')
    axs[i*2].grid(True, which='both')

    # Bar chart for transaction count by month for the current year
    axs[i*2 + 1].bar(monthly_summary_yearly['month'], monthly_summary_yearly['TransactionCount'], color='tab:red', alpha=0.6)
    axs[i*2 + 1].set_ylabel('Transaction Count')
    axs[i*2 + 1].grid(True, which='both')

# Setting x-ticks and labels for the last bar chart (shared x-axis for all)
axs[-1].set_xticks(range(1, 13))
axs[-1].set_xticklabels(months)
axs[-1].set_xlabel('Month')

plt.tight_layout()
plt.show()



In [ ]:
# Importing required libraries for visualization
import matplotlib.pyplot as plt
import pandas as pd

# Assuming 'transactions' DataFrame already contains 'year' and 'month' columns
# and is ready for visualization

# Getting a list of unique years and defining month labels
years = sorted(transactions['year'].unique())
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Adjusting the figure layout to place visualizations for each year next to each other
fig, axs = plt.subplots(2, len(years), figsize=(40 * len(years) / 2, 30), sharey='row', gridspec_kw={'height_ratios': [3, 1]})

for i, year in enumerate(years):
    # Filter transactions for the current year
    yearly_transactions = transactions[transactions['year'] == year]

    # Preparing data for the box plot: a list of amounts for each month for the current year
    amounts_per_month_yearly = [yearly_transactions[yearly_transactions['month'] == month]['amount'] for month in range(1, 13)]

    # Preparing data for the bar chart for the current year
    monthly_summary_yearly = yearly_transactions.groupby('month').agg(TotalAmount=('amount', 'sum'), TransactionCount=('amount', 'count')).reset_index()

    # Selecting the appropriate axes for multiple or single year scenarios
    ax_box = axs[0, i] if len(years) > 1 else axs[0]
    ax_bar = axs[1, i] if len(years) > 1 else axs[1]
    
    ax_box.boxplot(amounts_per_month_yearly, patch_artist=True)
    ax_box.set_title(f'{year} (Box Plot)')
    ax_box.set_yscale('symlog')
    ax_box.set_ylabel('Transaction Amounts (log scale)')
    ax_box.grid(True, which='both')

    ax_bar.bar(monthly_summary_yearly['month'], monthly_summary_yearly['TransactionCount'], color='tab:red', alpha=0.6)
    ax_bar.set_ylabel('Transaction Count')
    ax_bar.grid(True, which='both')

    # Setting common x-ticks and labels for all axes
    ax_bar.set_xticks(range(1, 13))
    ax_bar.set_xticklabels(months)

# Setting a common x-label for the entire figure
fig.text(0.5, 0.04, 'Month', ha='center')

plt.tight_layout()
plt.show()


In [ ]:
# Filtering transactions with balances below 0
negative_balances = transactions[transactions['balance'] < 0]

print(len(negative_balances))

plot_numerical_distributions(negative_balances, ['balance', 'amount'])


There appear to be 2999 transactions which have a negative balance, therefore after the transaction the account balance was negative. This implies that these accounts are in some kind of debt. 

## Loans

In [ ]:
loans = read_csv("data/loan.csv")

loans['date'] = pd.to_datetime(loans['date'], format='%y%m%d')

loans['status'] = loans['status'].map({
    "A": "Contract finished, no problems",
    "B": "Contract finished, loan not paid",
    "C": "Contract running, OK thus-far",
    "D": "Contract running, client in debt"
})

loans.rename(columns={
    'date': 'granted_date',
    'amount': 'amount',
    'duration': 'duration',
    'payments': 'monthly_payments',
    'status': 'status'
}, inplace=True)

loans.info()

In [ ]:
# todo add some basic eda here
loans.head()

In [ ]:
loans.describe()

In [ ]:
loans.nunique()

It seems as if one account can have at max one loan.

In [ ]:
plot_categorical_variables(loans, ['duration', 'status'])

The distribution of durations seems to be even.

In [ ]:
plot_numerical_distributions(loans, ['granted_date'])

## Credit Cards

In [ ]:
cards = read_csv("data/card.csv")

cards['issued'] = pd.to_datetime(cards['issued'], format='%y%m%d %H:%M:%S').dt.date

cards.info()

In [ ]:
cards.head()

In [ ]:
cards.describe()

In [ ]:
plot_categorical_variables(cards, ['type'])

In [ ]:
plot_numerical_distributions(cards, ['issued'])

## Demographic data

In [ ]:
districts = read_csv("data/district.csv")

# Rename columns
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
districts.rename(columns={
    'A1': 'district_id',
    'A2': 'district_name',
    'A3': 'region',
    'A4': 'inhabitants',
    'A5': 'small_municipalities',
    'A6': 'medium_municipalities',
    'A7': 'large_municipalities',
    'A8': 'huge_municipalities',
    'A9': 'cities',
    'A10': 'ratio_urban_inhabitants',
    'A11': 'average_salary',
    'A12': 'unemployment_rate_1995',
    'A13': 'unemployment_rate_1996',
    'A14': 'entrepreneurs_per_1000_inhabitants',
    'A15': 'crimes_committed_1995',
    'A16': 'crimes_committed_1996'
}, inplace=True)

for col in ['unemployment_rate_1995', 'unemployment_rate_1996', 'crimes_committed_1995', 'crimes_committed_1996']:
    districts[col] = pd.to_numeric(districts[col], errors='coerce')

districts.info()

It appears as if there is 1 null value for unemployment rate in 1995 and crimes committed in 1995.

In [ ]:
# todo add some basic eda here
districts.head()

In [ ]:
DeepnoteChart(districts, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"region","scale":{"type":"linear"},"stack":"zero"},"y":{"sort":null,"type":"quantitative","field":"crimes_committed_1995","scale":{"type":"linear"},"stack":"zero","format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"nominal","field":"cities","scale":{"scheme":"deepnote10"}}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{}}""")

In [ ]:
districts.describe()

In [ ]:
districts.nunique()

In [ ]:
plot_numerical_distributions(districts, ['crimes_committed_1995'])

In [ ]:
plot_categorical_variables(districts, ['region'])

We need to differentiate between the domicile of the client and account, as they can be different.


# Relationships

Following the documentation of the dataset, there are multiple relationships that need to be validated. https://sorry.vse.cz/~berka/challenge/PAST/index.html

The ERD according to the descriptions on https://sorry.vse.cz/~berka/challenge/PAST/index.html

[![](https://mermaid.ink/img/pako:eNqtV1Fv4jgQ_itWXu6l7SZ0gQatTsqGdhddCxVQrXSqFJnEgLWJnbOd7bGl__3GTgA3JGyvWh7ajPPNZ898nrHz7MQ8Ic7AIWJI8Urg7JEh-AVhOHkYz9FzaeqfVIKyFcJxzAumIpqg-7_QoxOUNhoNH50jcEL1Q2zQNxo9rAZq8AQrUv45EIaCYEU5Q0MYb-BeCvJPQVi8AZ-RlAVmMUE3u8Gdw0v5L7wdXTeGE6eUWNGExmxa3YIKtY5YkS2IAOBnbbYt7S1hVwsbjmb3k9loPpqMG1YHRPl-bUCSc0lNThqzfYjlpjWWYxVvfqmi2uRamMkTI-LDgySiFsNkOryeNqyei4SI_fIn2voNq1lg9j1SHGBTEtPcxPgZxk7w1tAVve2wTDlWCGdmWkg1WVBFEhRkdWDF_D2Sm2zBU8De402mWcM1FjhWRNCfZt_WkjSfBuNZELYIrQRmcp-qubaAq1XptyXMriqbsmXXVjJD3SVUffhG1ToR-AmnDVCeE1EW52vmO2gmJ9JqQ49TW4IXODWVDCVWPQVLSCqyfE_rYU_SpkltOxkVhYLt_YudZAFrm6hS-XYSNMkLkbG9urdg_C5ZDdcXiFjv1rqutewb7HHaKbxMir2cBjWs7GO2vNztEpB3nKl1ukFVAciGgKTCqpA71pmx6p05mA6b-jIWyaErg9F6wOR2lz3dIHc7XNPN4fkoqxTOEUjkbsbSstN66Nrz6SicN7fsfe-_P3HkVXCGM2KDxmA3wARZlfJMzUNdP8YjvowoW2PoW7iUZ8wvEF-i0WGw2SsrGIXOiFNIHJHRE9S9zRSlKuq6rtb7FRB9QjD8PkpwhJ4ceb7vH_PCy3P95n3UHbfk9hu59dtz_93kKxV5wNCQjD_NeDNrXEL2ioTGPi4tU3LaoRAL3S5eqfmgx2wx0VTDG8r9B7TmFYkkdE-h70ZBOYBmZuDYoWAky1NuajiCNZDI7-oJrWE9F0F_-N03efdavHvN6QGEILkgjBRCRnCwmBzXwr-2QQhAyGvPN88yqqAhRrGgGeho4gnNcz2IU14926tndYBaD9huz8_58_7KPDDtLsa6I4_2lVpHVxfSgSlpSROY5ADeUVVY-46oHQIpeUzNBPqU3nlVlP_PyYZpz-227Mja5SuWLSsqb3wa8wUk0bK3Ie1rTxlsDNfCZvS2PD4Hpv8y9BW2ssY5Z05GRIZpAt8ppuE-OmpNdJ_U0IQscZGaI-0FoLhQfLZhsTNQoiBnTpHrzl593DiDJU4ljOaY_c15tgPpGw8Xd-WnkPkiMhBn8Oz86wyuOhf9bsf96Lndvuu5ff_M2TgDz-1ddK763qXvel237131X86cn4bUvbi67PU6Xt_z3Y--e9ntv_wHEyE3kA?type=png)](https://mermaid.live/edit#pako:eNqtV1Fv4jgQ_itWXu6l7SZ0gQatTsqGdhddCxVQrXSqFJnEgLWJnbOd7bGl__3GTgA3JGyvWh7ajPPNZ898nrHz7MQ8Ic7AIWJI8Urg7JEh-AVhOHkYz9FzaeqfVIKyFcJxzAumIpqg-7_QoxOUNhoNH50jcEL1Q2zQNxo9rAZq8AQrUv45EIaCYEU5Q0MYb-BeCvJPQVi8AZ-RlAVmMUE3u8Gdw0v5L7wdXTeGE6eUWNGExmxa3YIKtY5YkS2IAOBnbbYt7S1hVwsbjmb3k9loPpqMG1YHRPl-bUCSc0lNThqzfYjlpjWWYxVvfqmi2uRamMkTI-LDgySiFsNkOryeNqyei4SI_fIn2voNq1lg9j1SHGBTEtPcxPgZxk7w1tAVve2wTDlWCGdmWkg1WVBFEhRkdWDF_D2Sm2zBU8De402mWcM1FjhWRNCfZt_WkjSfBuNZELYIrQRmcp-qubaAq1XptyXMriqbsmXXVjJD3SVUffhG1ToR-AmnDVCeE1EW52vmO2gmJ9JqQ49TW4IXODWVDCVWPQVLSCqyfE_rYU_SpkltOxkVhYLt_YudZAFrm6hS-XYSNMkLkbG9urdg_C5ZDdcXiFjv1rqutewb7HHaKbxMir2cBjWs7GO2vNztEpB3nKl1ukFVAciGgKTCqpA71pmx6p05mA6b-jIWyaErg9F6wOR2lz3dIHc7XNPN4fkoqxTOEUjkbsbSstN66Nrz6SicN7fsfe-_P3HkVXCGM2KDxmA3wARZlfJMzUNdP8YjvowoW2PoW7iUZ8wvEF-i0WGw2SsrGIXOiFNIHJHRE9S9zRSlKuq6rtb7FRB9QjD8PkpwhJ4ceb7vH_PCy3P95n3UHbfk9hu59dtz_93kKxV5wNCQjD_NeDNrXEL2ioTGPi4tU3LaoRAL3S5eqfmgx2wx0VTDG8r9B7TmFYkkdE-h70ZBOYBmZuDYoWAky1NuajiCNZDI7-oJrWE9F0F_-N03efdavHvN6QGEILkgjBRCRnCwmBzXwr-2QQhAyGvPN88yqqAhRrGgGeho4gnNcz2IU14926tndYBaD9huz8_58_7KPDDtLsa6I4_2lVpHVxfSgSlpSROY5ADeUVVY-46oHQIpeUzNBPqU3nlVlP_PyYZpz-227Mja5SuWLSsqb3wa8wUk0bK3Ie1rTxlsDNfCZvS2PD4Hpv8y9BW2ssY5Z05GRIZpAt8ppuE-OmpNdJ_U0IQscZGaI-0FoLhQfLZhsTNQoiBnTpHrzl593DiDJU4ljOaY_c15tgPpGw8Xd-WnkPkiMhBn8Oz86wyuOhf9bsf96Lndvuu5ff_M2TgDz-1ddK763qXvel237131X86cn4bUvbi67PU6Xt_z3Y--e9ntv_wHEyE3kA)

This ERD shows how the data appears in the dataset:

[![](https://mermaid.ink/img/pako:eNqtV99P2zAQ_lesvOyFbjCJSq2mSSHlRzRoUVq0F6TITdzWIrEz2xnqgP99ZydNTeIUhOgD5JzvPvvufJ-dJy_hKfHGHhETitcC5_cMwc8PgtnddIGeKlP_pBKUrRFOEl4yFdMU3f5C955f2Sic3HsdcEr1Q2LQFxo9qQda8BQrUv3ZEwaCYEU5QxMYd3CvBPlTEpZswSeUssQsIehiN7hzeKn-BdfhuTOcJKPEiiYwpmt1SyrUJmZlviQCgGfa7Fvae8KuFzYJ57ezebgIZ1PH6oCoaNYGJAWX1OTEme19LBe9sXSrePFmFdW20IWZPTIivt1JIloxzKLJeeRYPRcpEc3yZ9r6hNUsMXuIFQdYRBJamBjPYOwAbwtd09sOq4xjhXBupoVUkyVVJEV-3gbWzA-x3OZLngH2Fm9zzRpssMCJIoL-M_u2laRF5E_nftBTaCUwk02qFtoCrt5Kvy9hdlfZlD27ti4z9F1K1bffVG1SgR9x5oDygoiqOV8z34CYHEirDe2mtgIvcWY6GVqsfvJXkFRk-R6uhz1JX01a28lUUSjY3m_sJAvY2kR1la9nvqu8EBlrqnsNxmeV1XBdQsR6t7br2sq-wXbTTuFlWjblNKhJbXfZimq3S0DecKY22RbVDSAdAUmFVSl3rHNjtZXZjyYuXcYi3asyGL0HTGGr7GGB3O1wTbeA505WKZwjkMjdjJVlp3Wv2osoDBZuyW60__bAkVfDGc6JDZqC7YAJsq7KE5mHdv0Yj_kqpmyDQbdwVZ4p_4r4CoX7QbdXXjIKyogzSByR8SP0vc0UZyo-PT7W9X4FRD8QDH-MEhxBk-OT0WjU5YWXA_3mY9TfjyvukZNbvx2MPky-VvEJMDiS8dOMu1mTCtJUJDB2t7VMy2mHUiy1XLyq5p0es4uJIg13tPtfkOY1iSWop9B3I78aQHMz0HUoGcmLjJsejmENJB6d6gmtYT0XQV9Gp-_yHvZ4D93pAYQghSCMlELGcLCYHLfCP7dBCEDopD_fPM-pAkGME0FzqKOJJzDP7SAOeQ1tr6GlAC0NeH4eDPhTc2UeG7lLsFbksOnUNrq-kI5NS0uawiR78I6qxtp3RO3gS8kTaibQp_TOq6bUTs_P73WyYbWnUWTtcoVlz4qqG5_GXEJJdNn7kPa1pwo2gWuhEw1Tm-NzbPSXoSvYyhrnHXk5ETmmKXynGMG999SGaJ3U0JSscJmZI-0FoLhUfL5liTdWoiRHXlloZa8_brzxCmcSRvUVh4ub6tsn4WxF197LfwBRISI?type=png)](https://mermaid.live/edit#pako:eNqtV99P2zAQ_lesvOyFbjCJSq2mSSHlRzRoUVq0F6TITdzWIrEz2xnqgP99ZydNTeIUhOgD5JzvPvvufJ-dJy_hKfHGHhETitcC5_cMwc8PgtnddIGeKlP_pBKUrRFOEl4yFdMU3f5C955f2Sic3HsdcEr1Q2LQFxo9qQda8BQrUv3ZEwaCYEU5QxMYd3CvBPlTEpZswSeUssQsIehiN7hzeKn-BdfhuTOcJKPEiiYwpmt1SyrUJmZlviQCgGfa7Fvae8KuFzYJ57ezebgIZ1PH6oCoaNYGJAWX1OTEme19LBe9sXSrePFmFdW20IWZPTIivt1JIloxzKLJeeRYPRcpEc3yZ9r6hNUsMXuIFQdYRBJamBjPYOwAbwtd09sOq4xjhXBupoVUkyVVJEV-3gbWzA-x3OZLngH2Fm9zzRpssMCJIoL-M_u2laRF5E_nftBTaCUwk02qFtoCrt5Kvy9hdlfZlD27ti4z9F1K1bffVG1SgR9x5oDygoiqOV8z34CYHEirDe2mtgIvcWY6GVqsfvJXkFRk-R6uhz1JX01a28lUUSjY3m_sJAvY2kR1la9nvqu8EBlrqnsNxmeV1XBdQsR6t7br2sq-wXbTTuFlWjblNKhJbXfZimq3S0DecKY22RbVDSAdAUmFVSl3rHNjtZXZjyYuXcYi3asyGL0HTGGr7GGB3O1wTbeA505WKZwjkMjdjJVlp3Wv2osoDBZuyW60__bAkVfDGc6JDZqC7YAJsq7KE5mHdv0Yj_kqpmyDQbdwVZ4p_4r4CoX7QbdXXjIKyogzSByR8SP0vc0UZyo-PT7W9X4FRD8QDH-MEhxBk-OT0WjU5YWXA_3mY9TfjyvukZNbvx2MPky-VvEJMDiS8dOMu1mTCtJUJDB2t7VMy2mHUiy1XLyq5p0es4uJIg13tPtfkOY1iSWop9B3I78aQHMz0HUoGcmLjJsejmENJB6d6gmtYT0XQV9Gp-_yHvZ4D93pAYQghSCMlELGcLCYHLfCP7dBCEDopD_fPM-pAkGME0FzqKOJJzDP7SAOeQ1tr6GlAC0NeH4eDPhTc2UeG7lLsFbksOnUNrq-kI5NS0uawiR78I6qxtp3RO3gS8kTaibQp_TOq6bUTs_P73WyYbWnUWTtcoVlz4qqG5_GXEJJdNn7kPa1pwo2gWuhEw1Tm-NzbPSXoSvYyhrnHXk5ETmmKXynGMG999SGaJ3U0JSscJmZI-0FoLhUfL5liTdWoiRHXlloZa8_brzxCmcSRvUVh4ub6tsn4WxF197LfwBRISI)

In [ ]:
# Verify 1:1 relationships between CLIENT, LOAN and DISPOSITION
assert dispositions['client_id'].is_unique, "Each client_id should appear exactly once in the DISPOSITION DataFrame."
assert loans['account_id'].is_unique, "Each account_id should appear exactly once in the LOAN DataFrame."

# Verify 1:M relationships between ACCOUNT and DISPOSITION
#assert dispositions['account_id'].is_unique == False, "An account_id should appear more than once in the DISPOSITION DataFrame."
assert dispositions['account_id'].is_unique == True, "An account_id should appear once in the DISPOSITION DataFrame."
# TODO check if in accordance to decision to remove disponents from dispositions

# Verify each district_id in ACCOUNT and CLIENT exists in DISTRICT
assert set(accounts['district_id']).issubset(
    set(districts['district_id'])), "All district_ids in ACCOUNT should exist in DISTRICT."
assert set(clients['district_id']).issubset(
    set(districts['district_id'])), "All district_ids in CLIENT should exist in DISTRICT."

# Verify each account_id in DISPOSITION, ORDER, TRANSACTION, and LOAN exists in ACCOUNT
assert set(dispositions['account_id']).issubset(
    set(accounts['account_id'])), "All account_ids in DISPOSITION should exist in ACCOUNT."
assert set(orders['account_id']).issubset(
    set(accounts['account_id'])), "All account_ids in ORDER should exist in ACCOUNT."
assert set(transactions['account_id']).issubset(
    set(accounts['account_id'])), "All account_ids in TRANSACTION should exist in ACCOUNT."
assert set(loans['account_id']).issubset(
    set(accounts['account_id'])), "All account_ids in LOAN should exist in ACCOUNT."

# Verify each client_id in DISPOSITION exists in CLIENT
assert set(dispositions['client_id']).issubset(
    set(clients['client_id'])), "All client_ids in DISPOSITION should exist in CLIENT."

# Verify each disp_id in CARD exists in DISPOSITION
assert set(cards['disp_id']).issubset(set(dispositions['disp_id'])), "All disp_ids in CARD should exist in DISPOSITION."

In [ ]:
dispositions[dispositions["account_id"] == 3980]

In [ ]:
# are there any cards issued to dispositions that are not of type 'OWNER'?
cards[~cards['disp_id'].isin(dispositions[dispositions['type'] == "OWNER"]['disp_id'])]

In [ ]:
dispositions[dispositions['type'] != "OWNER"].head()

In [ ]:
# get all clients associated with account 3980
clients[clients['client_id'].isin(dispositions[dispositions['account_id'] == 3980]['client_id'])]

# Non-transactional Data

In [ ]:
orders_pivot = orders.pivot_table(index='account_id', 
                                 columns='k_symbol', 
                                 values='debited_amount',
                                 aggfunc='sum', 
                                 fill_value=0)

orders_pivot.columns = [f'k_symbol_debited_sum_{col.lower()}' for col in orders_pivot.columns]

# TODO: find something better than this
orders_pivot = orders_pivot.reset_index() # Use created index as account_id
orders_pivot.head()

In [ ]:
def merge_non_transactional_data(clients, districts, dispositions, accounts, orders, loans, cards):
    # Rename district_id for clarity in clients and accounts DataFrames
    clients = clients.rename(columns={'district_id': 'client_district_id'})
    accounts = accounts.rename(columns={'district_id': 'account_district_id'})
    
    # Prepare districts dataframe for merge with prefix for clients and accounts
    districts_client_prefixed = districts.add_prefix('client_')
    districts_account_prefixed = districts.add_prefix('account_')
    
    # Merge district information for clients and accounts with prefixed columns
    clients_with_districts = pd.merge(clients, districts_client_prefixed, left_on='client_district_id', right_on='client_district_id', how='left')
    accounts_with_districts = pd.merge(accounts, districts_account_prefixed, left_on='account_district_id', right_on='account_district_id', how='left')

    # Merge cards with dispositions and prefix card-related columns to avoid confusion
    cards_prefixed = cards.add_prefix('card_')
    dispositions_with_cards = pd.merge(dispositions, cards_prefixed, left_on='disp_id', right_on='card_disp_id', how='left')
    
    # Merge clients (with district info) with dispositions and cards
    # Assuming dispositions might have columns that overlap with clients, prefix those if necessary
    clients_dispositions_cards = pd.merge(dispositions_with_cards, clients_with_districts, on='client_id', how='left')
    
    # Merge the above with accounts (with district info) on account_id
    accounts_clients_cards = pd.merge(accounts_with_districts, clients_dispositions_cards, on='account_id', how='left')
    
    # Merge orders DataFrame, assuming orders might contain columns that could overlap, prefix as needed
    orders_prefixed = orders.add_prefix('order_')
    comprehensive_df_with_orders = pd.merge(accounts_clients_cards, orders_prefixed, left_on='account_id', right_on='order_account_id', how='left')
    
    # Merge loans with the comprehensive dataframe (now including orders) on account_id
    # Prefix loan-related columns to maintain clarity
    loans_prefixed = loans.add_prefix('loan_')
    final_df = pd.merge(comprehensive_df_with_orders, loans_prefixed, left_on='account_id', right_on='loan_account_id', how='left')

    final_df['account_created'] = pd.to_datetime(final_df['account_created'])
    final_df['card_issued'] = pd.to_datetime(final_df['card_issued'])

    return final_df

non_transactional_df = merge_non_transactional_data(clients, districts, dispositions, accounts, orders_pivot, loans, cards)
non_transactional_df.info()

In [ ]:
non_transactional_df.to_csv("./data/non_transactional.csv")

## EDA

### Card Holders

In [ ]:
plt.figure(figsize=(10, 6))
plt.title(f'Distribution of Age for Junior Card Holders\n total count = {len(non_transactional_df[non_transactional_df["card_type"] == "junior"])}')
sns.histplot(non_transactional_df[non_transactional_df['card_type'] == 'junior']['age'], kde=True, bins=30)
plt.xlabel('Age of Client (presumably in 1999)')
plt.plot()

Looking at the age distribution of Junior Card holders paints a picture on this group, however only looking at the current age may be misleading as we need to understand how old they were when the card was issued to determine if they could have been eligble for a Classic/Gold card (at least 18 when the card was issued). 

In [ ]:
non_transactional_df['card_issued'] = pd.to_datetime(non_transactional_df['card_issued'])

non_transactional_df['age_at_card_issuance'] = non_transactional_df['card_issued'] - non_transactional_df['birth_date']
non_transactional_df['age_at_card_issuance'] = non_transactional_df['age_at_card_issuance'].dt.days // 365

# plot the distribution of age at card issuance
plt.figure(figsize=(10, 6))
plt.title(f'Distribution of Age at Card Issuance for Junior Card Holders\n total count = {len(non_transactional_df[non_transactional_df["card_type"] == "junior"])}')
sns.histplot(non_transactional_df[non_transactional_df['card_type'] == 'junior']['age_at_card_issuance'], kde=True, bins=30)
plt.xlabel('Age at Card Issuance')
plt.plot()


Here we can see that roughly 1/3 of the Junior Card holders were not of legal age (assuming legal age is 18) when receiving their Junior Card.

In [ ]:

# POTENTIAL FIX FOR THE PLOT BELOW
plt.figure(figsize=(10, 6))
plt.title(f'Distribution of Age at Card Issuance for All Card Types\n total count = {len(non_transactional_df)}')
sns.histplot(non_transactional_df[non_transactional_df['card_type'] == 'junior']['age_at_card_issuance'], kde=True, bins=10, color='blue', label='Junior Card Holders')
sns.histplot(non_transactional_df[non_transactional_df['card_type'] != 'junior']['age_at_card_issuance'], kde=True, bins=30, color='red', label='Non-Junior Card Holders')
plt.legend()
plt.xlabel('Age at Card Issuance')
plt.plot()



Comparing the age at issue date between Junior and non-Junior (Classic/Gold) card holders shows that there is no overlap between the two groups, which makes intutively sense. 

Therefore removing the subset of Junior Cards seems as valid as there is no reason to believe that there are Junior Cards issued wrongly, the subset being relatively small compared to the remaining issued cards and the fact that our target is specifically Classic/Gold Card owners.  

Looking at the age distribution of Junior card holders and their occurence in comparison it seems valid to remove them as they are not the target group and make up a small subset of the complete dataset.

In [ ]:
# something is wrong here.

plt.figure(figsize=(10, 6))
plt.title('Number of Clients by Card Type')
sns.barplot(x=['No Card', 'Classic/Gold Card Holders', 'Junior Card Holders'], y=[non_transactional_df['card_type'].isna().sum(), non_transactional_df['card_type'].isin(['gold', 'classic']).sum(), non_transactional_df['card_type'].eq('junior').sum()])
# ensure that the number of clients is shown on the bars
for i, v in enumerate([non_transactional_df['card_type'].isna().sum(), non_transactional_df['card_type'].isin(['gold', 'classic']).sum(), non_transactional_df['card_type'].eq('junior').sum()]):
    plt.text(i, v + 10, str(v), ha='center', va='bottom')

plt.plot()

Looking at the distribution of card holders in general we can see that the most clients are not in a possession of a credit card. 


### Time factors on Card Status

The time between creating an account and issuing a card may also be important when filtering customers based on their history. We should avoid filtering out potentially interesting periods and understand how the timespans between account creation and card issuance are distributed.

In [ ]:
filtered_df['duration_days'] = (filtered_df['card_issued'] - filtered_df['account_created']).dt.days

plt.figure(figsize=(12, 8))
sns.histplot(filtered_df['duration_days'], bins=50, edgecolor='black', kde=True)
plt.title('Distribution of Duration Between Account Creation and Card Issuance')
plt.xlabel('Duration in Days')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

The histogram displays a distribution with multiple peaks, indicating that there are several typical time frames for card issuance after account creation. The highest peak occurs within the first 250 days, suggesting that a significant number of cards are issued during this period. The frequency decreases as duration increases, with noticeable peaks that may correspond to specific processing batch cycles or policy changes over time. The distribution also has a long tail, suggesting that in some cases, card issuance can take a very long time.

Analyzing the length of time a client has been with the bank in relation to their account creation date and card ownership can provide valuable insights for a bank's customer relationship management and product targeting strategies. Long-standing clients may exhibit different banking behaviors, such as product adoption and loyalty patterns, compared to newer clients.

In [ ]:
max_account_creation_date = non_transactional_df['card_issued'].max()

non_transactional_df['client_tenure_years_relative'] = (max_account_creation_date - non_transactional_df['account_created']).dt.days / 365.25

plt.figure(figsize=(10, 6))
ax = sns.histplot(
    data=non_transactional_df, 
    x='client_tenure_years_relative', 
    hue='has_card', 
    multiple='stack', 
    binwidth=1,
    stat="percent"
)

# Call the function to add labels
add_percentage_labels(ax, non_transactional_df['has_card'].unique())

# Additional plot formatting
plt.title('Client Tenure Relative to Latest Card Issued Date and Card Ownership')
plt.xlabel('Client Tenure (Years, Relative to Latest Card Issuance)')
plt.ylabel('Percentage of Clients')
plt.tight_layout()

# Display the plot
plt.show()

The bar chart shows the tenure of clients in years, categorized by whether they own a credit card (True) or not (False). Each bar represents the percentage of clients within a specific tenure range, allowing for comparison of the distribution of card ownership among clients with different lengths of association with the bank.



### Demographics

Using the available demographic data, we can investigate the potential correlation between demographic data and card status. The average salary may indicate a difference between cardholders and non-cardholders, as it is reasonable to assume that cardholders have a higher average salary than non-cardholders.

In [ ]:
non_transactional_df['has_card'] = ~non_transactional_df['card_card_id'].isna()

plt.figure(figsize=(10, 6))
sns.boxplot(x='has_card', y='client_average_salary', data=non_transactional_df)
plt.title("Average Salary in Client's Region by Card Ownership")
plt.xlabel('Has Card')
plt.ylabel('Average Salary')
plt.xticks([0, 1], ['No Card Owner', 'Card Owner'])

plt.tight_layout()
plt.show()


The box plot compares the average salaries of clients who own a credit card with those who do not. Both groups have a substantial overlap in salary ranges, suggesting that while there might be a trend for card owners to have higher salaries, the difference is not significant. The median salary for card owners is slightly higher than that for non-card owners, as indicated by the median line within the respective boxes.

Both distributions have outliers on the higher end, indicating that some individuals have salaries significantly above the average in both groups. However, these outliers do not dominate the general trend. 

It should also be noted that this plot assumes that the average salary of the region's clients remained constant over the years, which is unlikely to be true.

The group of bar charts represents the distribution of credit card ownership across various demographics, showing the percentage of clients with and without cards within different age groups, sexes, and regions.

In [ ]:
non_transactional_df['age_group'] = pd.cut(non_transactional_df['age'], bins=[0, 25, 40, 55, 70, 100], labels=['<25', '25-40', '40-55', '55-70', '>70'])

plt.figure(figsize=(15, 15))

# Age Group
plt.subplot(3, 1, 1)
age_group_counts = non_transactional_df.groupby(['age_group', 'has_card']).size().unstack(fill_value=0)
age_group_percentages = (age_group_counts.T / age_group_counts.sum(axis=1)).T * 100
age_group_plot = age_group_percentages.plot(kind='bar', stacked=True, ax=plt.gca())
age_group_plot.set_title('Card Ownership by Age Group')
age_group_plot.set_ylabel('Percentage')
add_percentage_labels(age_group_plot, non_transactional_df['has_card'].unique())

# Sex
plt.subplot(3, 1, 2)
sex_counts = non_transactional_df.groupby(['sex', 'has_card']).size().unstack(fill_value=0)
sex_percentages = (sex_counts.T / sex_counts.sum(axis=1)).T * 100
sex_plot = sex_percentages.plot(kind='bar', stacked=True, ax=plt.gca())
sex_plot.set_title('Card Ownership by Sex')
sex_plot.set_ylabel('Percentage')
add_percentage_labels(sex_plot, non_transactional_df['has_card'].unique())

# Client Region
plt.subplot(3, 1, 3)
region_counts = non_transactional_df.groupby(['client_region', 'has_card']).size().unstack(fill_value=0)
region_percentages = (region_counts.T / region_counts.sum(axis=1)).T * 100
region_plot = region_percentages.plot(kind='bar', stacked=True, ax=plt.gca())
region_plot.set_title('Card Ownership by Client Region')
region_plot.set_ylabel('Percentage')
region_plot.tick_params(axis='x', rotation=45)
add_percentage_labels(region_plot, non_transactional_df['has_card'].unique())

plt.tight_layout()
plt.show()


**Card Ownership by Age Group:**
The bar chart displays the proportion of cardholders in different age groups. The percentage of cardholders is lowest in the age group of over 70, followed by the age group of 55-70, indicating that card ownership is more prevalent among younger demographics.

**Card Ownership by Sex:**
The bar chart shows the breakdown of card ownership by sex. The data reveals that the percentage of cardholders is comparable between both sexes, and no significant difference is present.

**Card Ownership by Region**
The bar chart at the bottom illustrates card ownership across different regions, showing a relatively consistent pattern among most regions.

### Impact of Loans / Debt

In [ ]:
simplified_loan_status_mapping = {
    "Contract finished, no problems": "Finished",
    "Contract finished, loan not paid": "Not Paid",
    "Contract running, OK thus-far": "Running",
    "Contract running, client in debt": "In Debt",
    "No Loan": "No Loan"
}

non_transactional_df['loan_status_simplified'] = non_transactional_df['loan_status'].map(simplified_loan_status_mapping)

loan_status_simplified_card_ownership_counts = non_transactional_df.groupby(['loan_status_simplified', 'has_card']).size().unstack(fill_value=0)
loan_status_simplified_card_ownership_percentages = (loan_status_simplified_card_ownership_counts.T / loan_status_simplified_card_ownership_counts.sum(axis=1)).T * 100

loan_status_simplified_card_ownership_percentages.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Interaction Between Simplified Loan Status and Card Ownership')
plt.xlabel('Simplified Loan Status')
plt.ylabel('Percentage of Clients')
plt.xticks(rotation=45)
plt.legend(title='Has Card', labels=['No Card', 'Has Card'])
plt.tight_layout()
plt.show()


# Transactional Data

In [ ]:
zero_amount_transactions = transactions[transactions['amount'] == 0]

zero_amount_transactions_info = {
    'total_zero_amount_transactions': len(zero_amount_transactions),
    'unique_accounts_with_zero_amount': zero_amount_transactions['account_id'].nunique(),
    'transaction_type_distribution': zero_amount_transactions['transaction_type'].value_counts(normalize=True),
    'operation_distribution': zero_amount_transactions['operation'].value_counts(normalize=True),
    'k_symbol_distribution': zero_amount_transactions['k_symbol'].value_counts(normalize=True)
}

zero_amount_transactions_info, len(zero_amount_transactions_info)

In [ ]:
accounts_with_zero_amount_transactions = zero_amount_transactions['account_id'].unique()
accounts_features = accounts[accounts['account_id'].isin(accounts_with_zero_amount_transactions)]

# Summary of account features (e.g., account creation date distribution)
accounts_features


## Aggregate Transactions on a Monthly Basis

Validating first transactions where the amount equals the balance is essential for the integrity of our aggregated data analysis. This specific assertion underpins the reliability of our subsequent aggregation operations by ensuring each account's financial history starts from a verifiable point.

In [ ]:
def validate_first_transactions(transactions):
    """
    Validates that for each account in the transactions DataFrame, there is at least
    one transaction where the amount equals the balance on the account's first transaction date.

    Parameters:
    - transactions (pd.DataFrame): DataFrame containing transaction data with columns
      'account_id', 'date', 'amount', and 'balance'.

    Raises:
    - AssertionError: If not every account has a first transaction where the amount equals the balance.
    """

    first_dates = transactions.groupby('account_id')['date'].min().reset_index(name='first_date')

    first_trans = pd.merge(transactions, first_dates, how='left', on=['account_id'])

    first_trans_filtered = first_trans[(first_trans['date'] == first_trans['first_date']) & (first_trans['amount'] == first_trans['balance'])]

    first_trans_filtered = first_trans_filtered.drop_duplicates(subset=['account_id'])

    unique_accounts = transactions['account_id'].nunique()
    assert unique_accounts == first_trans_filtered['account_id'].nunique(), "Not every account has a first transaction where the amount equals the balance."

    return "Validation successful: Each account has a first transaction where the amount equals the balance."

validate_first_transactions(transactions)

We can confirm the truth of the assertions made. It is certain that there is a transaction with an amount equal to the balance in the transaction history of any account on the first date.

The function `aggregate_transactions_monthly` is designed to process and summarize financial transactions on a monthly basis for each account within a dataset. The explanation of its workings, step by step, is as follows:

1. **Sorting Transactions**: Initially, the function sorts the transactions in the provided DataFrame `df` based on `account_id` and the transaction `date`. This ensures that all transactions for a given account are ordered chronologically, which is crucial for accurate monthly aggregation and cumulative balance calculation.

2. **Monthly Grouping**: Each transaction's date is then converted to a monthly period using `dt.to_period("M")`. This step categorizes each transaction by the month and year it occurred, facilitating the aggregation of transactions on a monthly basis.

3. **Aggregation of Monthly Data**: The function groups the sorted transactions by `account_id` and the newly created `month` column. For each group, it calculates several metrics:
   - `volume`: The sum of all transactions' amounts for the month, representing the total money flow.
   - `total_abs_amount`: The sum of the absolute values of the transactions' amounts, indicating the total amount of money moved, disregarding the direction.
   - `transaction_count`: The count of transactions, providing a sense of activity level.
   - `positive_transaction_count` and `negative_transaction_count`: The counts of positive (inflows) and negative (outflows) transactions, respectively. This distinction can help identify the balance between income and expenses.
   - Statistical measures like `average_amount`, `median_amount`, `min_amount`, `max_amount`, and `std_amount` offer insights into the distribution of transaction amounts.
   - `type_count`, `operation_count`, and `k_symbol_count`: The counts of unique transaction types, operations, and transaction symbols (k_symbol), respectively, indicating the diversity of transaction characteristics.

4. **Cumulative Balance Calculation**: After aggregating the monthly data, the function computes a cumulative balance (`balance`) for each account by cumulatively summing the `volume` (total transaction amount) over time. This step provides insight into how the account balance evolves over the months.

5. **Validation of Aggregated Data**: The `validate_monthly_aggregated_transactions` function is invoked to ensure the integrity and correctness of the aggregated data through several assertions:
   - The balance should consistently increase or decrease based on whether the total monthly transaction volume is positive or negative, respectively.
   - For each account, the balance in the first month should equal the total transaction volume of that month.
   - The sum of positive and negative transaction counts must equal the total transaction count for each month.
   - The number of unique accounts in the aggregated data should match that in the original dataset.
   - The final balances of accounts in the aggregated data should closely match their last recorded transactions in the original dataset.

In [ ]:
def aggregate_transactions_monthly(df):
    """
    Aggregate financial transaction data on a monthly basis per account.

    Parameters:
    - df (pd.DataFrame): DataFrame containing financial transaction data with 'account_id', 'date', and other relevant columns.

    Returns:
    - pd.DataFrame: Monthly aggregated financial transaction data per account.
    """
    df_sorted = df.sort_values(by=["account_id", "date"])
    df_sorted["month"] = df_sorted["date"].dt.to_period("M")

    monthly_aggregated_data = (
            df_sorted.groupby(["account_id", "month"])
            .agg(
                volume=("amount", "sum"),
                total_abs_amount=("amount", lambda x: x.abs().sum()),
                transaction_count=("amount", "count"),
                positive_transaction_count=("amount", lambda x: (x >= 0).sum()), # TODO: it seems that there are some transactions with 0 amount, how to handle those?
                negative_transaction_count=("amount", lambda x: (x < 0).sum()),
                average_amount=("amount", "mean"),
                median_amount=("amount", "median"),
                min_amount=("amount", "min"),
                max_amount=("amount", "max"),
                std_amount=("amount", "std"),
                type_count=("transaction_type", "nunique"),
                operation_count=("operation", "nunique"),
                k_symbol_count=("k_symbol", "nunique"),
            )
            .reset_index()
            .sort_values(by=["account_id", "month"])
        )

    monthly_aggregated_data["balance"] = monthly_aggregated_data.groupby("account_id")["volume"].cumsum()

    validate_monthly_aggregated_transactions(monthly_aggregated_data, df)

    return monthly_aggregated_data

def validate_monthly_aggregated_transactions(aggregated_data, original_df):
    """
    Validate the integrity and correctness of aggregated monthly financial transactions.

    Parameters:
    - aggregated_data (pd.DataFrame): Aggregated monthly transaction data.
    - original_df (pd.DataFrame): Original dataset of financial transactions.

    Raises:
    - AssertionError: If validation conditions are not met.
    """
    
    assert (aggregated_data["volume"] >= 0).all() == (
        aggregated_data["balance"].diff() >= 0
    ).all(), "If the total amount is positive, the balance should go up."

    assert (aggregated_data["volume"] < 0).all() == (
        aggregated_data["balance"].diff() < 0
    ).all(), "If the total amount is negative, the balance should go down."

    first_month = aggregated_data.groupby("account_id").nth(0)
    assert (
        first_month["volume"] == first_month["balance"]
    ).all(), "The balance should equal the volume for the first month."

    assert (
        aggregated_data["positive_transaction_count"]
        + aggregated_data["negative_transaction_count"]
        == aggregated_data["transaction_count"]
    ).all(), "The sum of positive and negative transaction counts should equal the total transaction count."
    
    assert (
        aggregated_data["account_id"].nunique() == original_df["account_id"].nunique()
    ), "The number of unique account_ids in the aggregated DataFrame should be the same as the original DataFrame."

    assert (
        pd.merge(
            aggregated_data.groupby("account_id")
            .last()
            .reset_index()[["account_id", "balance"]],
            original_df[original_df.groupby("account_id")["date"].transform("max") == original_df["date"]][
                ["account_id", "balance"]
            ],
            on="account_id",
            suffixes=("_final", "_last"),
        )
        .apply(
            lambda x: np.isclose(x["balance_final"], x["balance_last"], atol=5), axis=1
        )
        .any()
    ), "Some accounts' final balances do not match their last transactions."
    

transactions_monthly = aggregate_transactions_monthly(transactions)
validate_monthly_aggregated_transactions(transactions_monthly, transactions)
transactions_monthly.describe()

### Monthly Balance Difference and Volume

This plot gives a clear picture of how money moves in and out of an account each month and how these movements affect the overall balance. It does this by showing two things:

- **Balance Difference**: This line shows whether the account balance went up or down each month. If the line goes up, it means the account gained money that month. If it goes down, the account lost money.
- **Volume**: This line shows the total amount of money that moved in the account each month, regardless of whether it was coming in or going out.

**What to Look For**:
- A direct link between the amount of money moved (volume) and changes in the account balance. High incoming money should lead to an uptick in the balance, and lots of outgoing money should lead to a downturn.
- This visual check helps to understand how active the account is and whether it’s generally getting fuller or emptier over time.

In [ ]:
def plot_monthly_balance_diff_and_volume(transactions_monthly, account_id, figsize=(12, 8)):
    account_transactions = transactions_monthly[transactions_monthly['account_id'] == account_id].sort_values(by='month')
    account_transactions['balance_diff'] = account_transactions['balance'].diff()

    plt.figure(figsize=figsize)

    plt.plot(account_transactions['month'].astype(str), account_transactions['balance_diff'], marker='o', label='Balance Difference')
    plt.plot(account_transactions['month'].astype(str), account_transactions['volume'], marker='x', linestyle='--', label='Volume')

    plt.title(f'Monthly Balance Difference and Volume for Account {account_id}')
    plt.xlabel('Month')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_monthly_balance_diff_and_volume(transactions_monthly, 2)

### Monthly Transactions, Balance, and Volume Plot Explanation

This visualization offers a snapshot of an account’s activity over time by comparing money movement each month with the overall account balance. It helps to understand:

- **Volume**: How much money came in or went out of the account each month. Incoming money is shown as up, and outgoing money as down.
- **Balance**: The total money in the account at the end of each month, showing how it's changed over time due to the monthly transactions.

**What to Look For**:
- How the monthly money movement impacts the account's growing or shrinking balance. For example, a few months of high income should visibly increase the balance.
- This simple visual guide helps spot trends, like if the account is steadily growing, holding steady, or facing issues, giving quick insights into financial well-being and further validates the aggregation made in the previous step.

In [ ]:
def plot_monthly_transactions_balance_and_volume(transactions_monthly, account_id):
    account_transactions = transactions_monthly[transactions_monthly['account_id'] == account_id]

    plt.figure(figsize=(12, 8))

    plt.plot(account_transactions['month'].astype(str), account_transactions['volume'], marker='o', label='Volume')
    plt.plot(account_transactions['month'].astype(str), account_transactions['balance'], marker='x', linestyle='--', label='Balance')

    plt.title(f'Monthly Transactions and Balance for Account {account_id}')
    plt.xlabel('Month')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_monthly_transactions_balance_and_volume(transactions_monthly, 2)

## Set artificial issue date for non-card holders

In [ ]:
def add_months_since_account_to_card(df):
    df['months_since_account_to_card'] = df.apply(
        lambda row: (row['card_issued'].to_period('M') - row['account_created'].to_period('M')).n
        if pd.notnull(row['card_issued']) and pd.notnull(row['account_created']) else np.nan, axis=1)
    return df

def filter_clients_without_sufficient_history(non_transactional_df, min_history_months=25):
    if 'months_since_account_to_card' not in non_transactional_df.columns:
        print("Warning: months_since_account_to_card column not found. Calculating history length.")
        non_transactional_df = add_months_since_account_to_card(non_transactional_df)

    count_before = len(non_transactional_df)
    filtered_df = non_transactional_df[non_transactional_df['months_since_account_to_card'].isnull() | (non_transactional_df['months_since_account_to_card'] >= min_history_months)]
    print(f"Filtered out {count_before - len(filtered_df)} records with less than {min_history_months} months of history. Percentage: {(count_before - len(filtered_df)) / count_before * 100:.2f}%.")
    return filtered_df

filtered_non_transactional_df = filter_clients_without_sufficient_history(non_transactional_df)

In [ ]:
non_transactional_df_w_card = filtered_non_transactional_df.dropna(subset=['card_issued']).copy()

plt.figure(figsize=(12, 8))
sns.histplot(non_transactional_df_w_card['months_since_account_to_card'], kde=True, bins=30)
plt.title('Distribution of Months from Account Creation to Card Issuance')
plt.xlabel('Months')
plt.ylabel('Count')
plt.grid(True)
plt.tight_layout()
plt.show()

The following approaches are in discussion:
- Looking at the distributions above extract the amount of history a buyer most likely has at the issue data of the card
- For each non buyer, find a buyer which was active in a similar time window (Jaccard similarity on the Year-Month sets)

### Match by similar transaction activity

#### NEW IDEA

Instead of looking at the full activity of a buyer, we only look at the pre-purchase activity as there is reason to believe that clients may change their patterns after purchasing date and therefore add unwanted bias. 

![](./docs/IMG_BBEF82A6C6B5-1.jpeg)

In [ ]:
from sklearn.metrics import pairwise_distances

def is_match_eligible(non_card_holder, card_holder):
    # Ensure the card issue date is after the non_card_holder's account creation
    if card_holder['card_issued'] <= non_card_holder['account_created']:
        return False
    
    # Ensure the card issue date is at least 25 months after the non_card_holder's account creation
    # This way we check if the fictional card issue date is at least 25 months after the account creation
    months_since_account_to_card = (card_holder['card_issued'].to_period('M') - non_card_holder['account_created'].to_period('M')).n
    if months_since_account_to_card < 25:
        return False
    
    return True

def match_non_card_holders_to_card_holders(non_transactional, transactions_df, top_n=5):
    non_transactional_df_w_card = non_transactional[non_transactional['card_issued'].notna()]
    non_transactional_df_wo_card = non_transactional[non_transactional['card_issued'].isna()]

    assert len(non_transactional_df_w_card) + len(non_transactional_df_wo_card) == len(non_transactional), "The split of non_transactional_df_w_card and non_transactional_df_wo_card should cover all records."

    transactions_df_monthly = transactions_df.copy()
    transactions_df_monthly['month_year'] = transactions['date'].dt.to_period('M')
    transactions_df_monthly['active'] = 1

    account_activity = transactions_df_monthly.pivot_table(index='account_id', columns='month_year', values='active', fill_value=0)
    account_activity.columns = [f'active_{col}' for col in account_activity.columns]
    
    card_records_w_activity = non_transactional_df_w_card.join(account_activity, on='account_id')
    non_card_records_w_activity = non_transactional_df_wo_card.join(account_activity, on='account_id')
    
    # assert number of columns with active_ prefix is the same for both DataFrames
    active_card_columns = card_records_w_activity.filter(like='active').columns
    active_non_card_columns = non_card_records_w_activity.filter(like='active').columns
    assert len(active_card_columns) == len(active_non_card_columns), "The number of columns with the active_ prefix should be the same for both DataFrames."

    client_id_issue_date = {}
    matches = []
    
    for index, non_card_holder in non_card_records_w_activity.iterrows():
        non_card_holder_vector = non_card_holder.filter(like='active').astype(bool).values.reshape(1, -1)
        card_holder_matrix = card_records_w_activity.filter(like='active').astype(bool).values
    
        distances = pairwise_distances(non_card_holder_vector, card_holder_matrix, metric='jaccard').flatten()

        eligible_indices = [i for i, distance in enumerate(distances) if is_match_eligible(non_card_holder, card_records_w_activity.iloc[i])]
        
        if len(eligible_indices) > 0:
            sorted_eligible_indices = sorted(eligible_indices, key=lambda i: distances[i])
            top_eligible_indices = sorted_eligible_indices[:top_n]
            selected_index = np.random.choice(top_eligible_indices)
            similarity_index = 1 - distances[selected_index]
            
            most_similar_card_holder = card_records_w_activity.iloc[selected_index]
            client_id_issue_date[non_card_holder['client_id']] = most_similar_card_holder['card_issued']
            matches.append((non_card_holder['client_id'], most_similar_card_holder['client_id'], similarity_index))
        else:
            print(f"No matches found for client_id {non_card_holder['client_id']}.")
        
    return client_id_issue_date, matches

client_id_issue_date, matches = match_non_card_holders_to_card_holders(filtered_non_transactional_df, transactions_monthly)
print(f"Percentage of non-card holders matched: {len(client_id_issue_date) / len(filtered_non_transactional_df[filtered_non_transactional_df['card_issued'].isna()]) * 100:.2f}%")

In [ ]:
def add_artificial_card_issued_column(non_transactional_df, client_id_issue_date):
    non_transactional_df_augmented = non_transactional_df.copy()
    
    non_transactional_df_augmented['has_card'] = True
    #non_transactional_df_augmented['card_issued'] = non_transactional_df_augmented['client_id'].map(client_id_issue_date)
    for client_id, issue_date in client_id_issue_date.items():
        non_transactional_df_augmented.loc[non_transactional_df_augmented['client_id'] == client_id, 'card_issued'] = issue_date
        non_transactional_df_augmented.loc[non_transactional_df_augmented['client_id'] == client_id, 'has_card'] = False

    count_before = len(non_transactional_df_augmented)
    non_transactional_df_augmented = non_transactional_df_augmented[non_transactional_df_augmented['card_issued'].notna()]
    print(f"Filtered out {count_before - len(non_transactional_df_augmented)} records with no card issued. Percentage: {(count_before - len(non_transactional_df_augmented)) / count_before * 100:.2f}%.")
    
    print(non_transactional_df_augmented['has_card'].value_counts())
        
    return non_transactional_df_augmented

count_nan_before = filtered_non_transactional_df['card_issued'].isna().sum()
filtered_non_transactional_df = add_artificial_card_issued_column(filtered_non_transactional_df, client_id_issue_date)

print(f"Percentage of NaN values in card_issued column before: {count_nan_before / len(filtered_non_transactional_df) * 100:.2f}%")
print(f"Percentage of NaN values in card_issued column after: {filtered_non_transactional_df['card_issued'].isna().sum() / len(filtered_non_transactional_df) * 100:.2f}%")

## Rolling Window Aggregations for Card Holders

### Function Overview

The `process_cardholder_transactions` function is designed to process and aggregate the transactional data for cardholders based on the history length from account creation to card issuance.

### Steps

1. **Filtering Card Issued Records**: Initially, the function filters out all records from a non-transactional dataset where no card has been issued (`card_issued` is NaN). This ensures that subsequent operations are only performed on accounts where a card has indeed been issued.

2. **Calculating History Length**: For each cardholder in the filtered dataset, the function calculates the history length in months from the account creation date (`account_created`) to the card issuance date (`card_issued`). This calculation helps in identifying the duration for which the account has been active before a card was issued.

3. **Identifying Eligible Cardholders**: The function then filters out cardholders based on a minimum history length criterion, specified by `min_history_months`. Only cardholders who have an account history equal to or longer than this threshold are considered eligible for further analysis.

4. **Merging Transaction Data**: The eligible cardholders' information is merged with monthly transaction data based on `account_id`. This step associates each transaction with the corresponding cardholder's account information, preparing the data for detailed transaction analysis.

5. **Assigning Card Issuance Date to Transactions**: For each transaction in the merged dataset, the function assigns the card issuance date from the cardholder's information. This is crucial for calculating the period before card issuance during which the transactions occurred.

6. **Filtering Transactions Before Card Issuance**: Transactions are filtered based on the number of months before card issuance, defined by the `months_before_card_range` parameter. This step ensures that only transactions occurring within the specified period before card issuance are considered for aggregation.

7. **Aggregating Transactions**: The function aggregates the filtered transactions by `account_id` and the number of months before card issuance. Aggregation metrics include sum, mean, median, min, max, and standard deviation for various transaction attributes like volume, absolute amount, transaction counts, and balance. This aggregation provides a comprehensive overview of spending patterns.

8. **Pivoting Aggregated Data**: The aggregated transaction data is pivoted to create a wide-format DataFrame where each row represents an account, and columns represent aggregated transaction metrics for different months before card issuance. This format is useful for analyzing trends over time leading up to card issuance.

9. **Final Dataset Preparation**: Finally, the function resets the index of the pivoted DataFrame, ensuring that `account_id` is a column in the final dataset, which makes it easier to identify each account's aggregated transaction metrics.

In [ ]:
def pivot_transactions(non_transactional_df, transactions_monthly_df, months_before_card_range=(2, 13)):
    merged_df = transactions_monthly_df.merge(non_transactional_df[['account_id']], on='account_id')

    merged_df['card_issued_date'] = merged_df['account_id'].map(non_transactional_df.set_index('account_id')['card_issued'])
    merged_df['months_before_card'] = merged_df.apply(lambda row: (row['card_issued_date'].to_period('M') - row['month']).n, axis=1)

    start_month, end_month = months_before_card_range
    filtered_df = merged_df.query(f"{start_month} <= months_before_card <= {end_month}")
    
    aggregated_data = filtered_df.groupby(['account_id', 'months_before_card']).agg({
        'volume': 'sum',
        'total_abs_amount': 'sum',
        'transaction_count': 'sum',
        'positive_transaction_count': 'sum',
        'negative_transaction_count': 'sum',
        'average_amount': 'mean',
        'median_amount': 'median',
        'min_amount': 'min',
        'max_amount': 'max',
        'std_amount': 'std',
        'type_count': 'sum',
        'operation_count': 'sum',
        'k_symbol_count': 'sum',
        'balance': 'mean'
    }).reset_index()

    pivoted_data = aggregated_data.pivot(index='account_id', columns='months_before_card')
    pivoted_data.columns = ['_'.join(['M', str(col[1]), col[0]]) for col in pivoted_data.columns.values]

    final_dataset = pivoted_data.reset_index()
    return final_dataset

transactions_pivoted = pivot_transactions(filtered_non_transactional_df, transactions_monthly)
transactions_pivoted.describe()

# Merge everything together

In [ ]:
final_df = filtered_non_transactional_df.merge(transactions_pivoted, on='account_id', how='left')
assert final_df['client_id'].is_unique, "Each client_id should appear exactly once in the final DataFrame."
assert final_df['account_id'].is_unique, "Each account_id should appear exactly once in the final DataFrame."

In [ ]:
final_df.describe()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Number of Clients by Card Issuance Status')
sns.countplot(x='has_card', data=final_df)
plt.xlabel('Card Issued')
plt.ylabel('Count')
plt.plot()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Distribution of Card Issuance Dates')
sns.histplot(final_df, x='card_issued', hue='has_card', kde=True, bins=30)
plt.xlabel('Card Issuance Date')
plt.ylabel('Count')
plt.plot()

# Model Construction

# Feature Engineering

# Model Engineering

# Model Comparison & Selection

# WORKBENCH - remove later

### Some older input

We need some categorical indicator wheter a transactions is a transactions incoming or outgoing from the perspective of the account holder. This will be important for the feature engineering later on. We will create a column called `transaction_direction` using the amount to engineer this feature.

Balance is the wealth on the account after the transaction.

k_symbol is the purpose of the transaction. This is often use in the context of budgeting in E-Banking applications or just personal finance management. A lot of NA values are present in this column. We will have to deal with this later on and weigh the importance of this column.

Track the time series of a given account to get a better understanding of the datasets nature.

It seems that there can be multiple transactions on the same day. We will have to aggregate the transactions on the same day to get a better understanding of the transactions as the timestamp resolution is not high enough to track the transactions on a daily basis.

We need some handling for this as the ID is not informative as well (Dani). 

For the feature enginnering a per month evaluation of the transactions is sufficient (Dani). 

We need to make sure across the board that for the prediction we only use the data that is available at the time of the prediction. This means that we can only use the data from the past to predict the future. This is important to keep in mind when we engineer the features as some entities do not have any information about the date and therefore we cannot use them for the prediction as we cannot rule out that they are not from the future.

Frequency analysis of the transactions could be interesting as the hypothesis might be that the more frequent the transactions the more likely the account holder is to be interested in a credit card. Fourier transformation could be used to get a better understanding of the frequency of the transactions.

### JITT 05.03.24
- New customers are handled differently
- Customer without the required history should be ignored otherwise they are treated as irrelevant
- Lag is ignored like (12 + 1) months
- Age should be in relation to the time of the event (card issued / reference date for refrence clients)
- How old are customers with a Junior Card? This should be evaluated based on the data
    - Example with Junior Card model with Age as most important feature as a negative example
- Reference clients
    - They should not be as similar as possible (Twin brother problem)
    - Same external market conditions
    - Same environment
    - See slide 6
- Owner and disponents cannot be distinguished directly and assumptions are required
    - MasterCards vs Visa war: as much cards as possible for both client of an account
    - AGain the Twin brother problem as features are too similar possibly

#### General notes to self
- Visualise monthly product puchases
- Viz environment of selected client and reference clients and answer the questions are they from a comparable environment

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7d865ccc-7b5c-4f8d-b4e6-4e008791345d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>